## 基础信息及配置

In [1]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))

sys.path.extend(['./cnn_finetune','./lib'])

from load_cifar import load_cifar10_data, load_cifar100_data

import numpy as np; print('numpy:', np.__version__)
import sklearn; print('sklearn:', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import keras; print('keras', keras.__version__)

import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

from lib.DataGenerator import DataGenerator

# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

Python 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609] on linux


Using TensorFlow backend.


numpy: 1.16.2
sklearn: 0.20.3
tensorflow 1.13.1
keras 2.2.4


In [2]:
config = tf.ConfigProto()

# cpu
# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores, inter_op_parallelism_threads=num_cores, allow_soft_placement=True, device_count={'CPU': 4})
    
# gpu allow_growth
config.gpu_options.allow_growth = True

# set memory limit of gpu
# config.gpu_options.per_process_gpu_memory_fraction = 0.5

set_session(tf.Session(config=config))

## 加载模型

In [3]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger


from cnn_finetune.resnet_152 import resnet152_model

In [4]:
batch_size = 16

In [5]:
def get_model(model_name):
    img_rows, img_cols = 224, 224 # Resolution of inputs
    channel = 3
    num_classes = 2019
    
    load_weights = False
    fine_tuning = False
    
    # Load our model
    model = model_name(img_rows, img_cols, channel, 
                              num_classes=num_classes, fine_tuning=fine_tuning, load_weights=load_weights)
    return model

## 运用val数据集进行验证

In [6]:
def validate_val_dataset(model):
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])
    
    # load val dataset
    val_data_generator = DataGenerator(file_type='val', corp=True, batch_size=batch_size, shuffle=False)
    
    # evaluate the model
    valid_evaluate = model.evaluate_generator(val_data_generator, 
                                     steps = len(val_data_generator),
                                     workers=30, use_multiprocessing=True,
                                     verbose=1)
    print(valid_evaluate)
    return valid_evaluate

In [14]:
def validate_model(model, data_generator):
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])

    # evaluate the model
    evaluate_info = model.evaluate_generator
    
    print(evaluate_info)
    return evaluate_info

## 预测test数据集

In [7]:
def predict(model, data_generator, steps):
    predictions = model.predict_generator(data_generator, 
                                     steps = steps,
                                     workers=30, use_multiprocessing=True,
                                     verbose=1)
    filenames = data_generator.image_iterator.filenames
    
    return filenames, predictions

## 预测过程

In [8]:
!ls -l models

total 1259776
drwxr-xr-x 3 root root      4096 May 24 18:04 darknet53
-rw-r--r-- 1 root root 127695716 May  8 08:03 denesnet_161_20190508_2.9911_0.3629.hd5
-rw-r--r-- 1 root root 127695716 May 12 12:29 denesnet_161_4.76-0.18-model.hd5
-rw-r--r-- 1 root root 127393236 May 12 12:29 denesnet_161_4.76-0.18-weights.hd5
-rw-r--r-- 1 root root  33552480 May  7 15:29 densenet121_weights_tf.h5
drwxr-xr-x 4 root root      4096 May 14 03:44 densenet161
drwxr-xr-x 3 root root      4096 May 12 04:49 densenet161_0512_val_loss_wrong
drwxr-xr-x 3 root root      4096 May 14 19:21 densenet161_former_3_GPU
drwxr-xr-x 3 root root      4096 May  9 16:32 densenet161_train_1
drwxr-xr-x 4 root root      4096 May  8 12:40 densenet161_val
drwxr-xr-x 3 root root      4096 May 10 00:39 densenet161_val_2
drwxr-xr-x 3 root root      4096 May 10 01:21 densenet161_val_3
drwxr-xr-x 3 root root      4096 May 10 08:48 densenet161_verifiy
-rw-r--r-- 1 root root 117815848 May  7 15:29 densenet161_weights_tf.h5
-rw-r--r-- 

In [9]:
!ls -l models/resnet_152

total 984412
-rw-r--r-- 1 root root 252004100 May 22 07:34 imaterialist-best.hd5
-rw-r--r-- 1 root root 252004100 May 21 16:49 imaterialist-e01-5.4681.hd5
-rw-r--r-- 1 root root 252004100 May 22 00:07 imaterialist-e02-3.9481.hd5
-rw-r--r-- 1 root root 252004100 May 22 07:34 imaterialist-e03-2.9979.hd5
-rw-r--r-- 1 root root         0 May 22 11:48 imaterialist-log.csv
drwxr-xr-x 2 root root      4096 May 22 11:52 logs


### densenet 161 

In [10]:
from cnn_finetune.densenet161_weights_path import densenet161_model

densenet161_predict_model = get_model(densenet161_model)

Instructions for updating:
Colocations handled automatically by placer.
dense161_model: classes: 2019 fine-tuning: False
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2 (None, 230, 230, 3)  0           data[0][0]                       
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 96) 14112       conv1_zeropadding[0][0]          
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 96) 384         conv1[0][0]           

__________________________________________________________________________________________________
conv5_12_x1_scale (Scale)       (None, 7, 7, 1584)   3168        conv5_12_x1_bn[0][0]             
__________________________________________________________________________________________________
relu5_12_x1 (Activation)        (None, 7, 7, 1584)   0           conv5_12_x1_scale[0][0]          
__________________________________________________________________________________________________
conv5_12_x1 (Conv2D)            (None, 7, 7, 192)    304128      relu5_12_x1[0][0]                
__________________________________________________________________________________________________
conv5_12_x2_bn (BatchNormalizat (None, 7, 7, 192)    768         conv5_12_x1[0][0]                
__________________________________________________________________________________________________
conv5_12_x2_scale (Scale)       (None, 7, 7, 192)    384         conv5_12_x2_bn[0][0]             
__________

In [11]:
weights_path="./models/temp_submit/imaterialist-e01-1.8070.hd5"
densenet161_predict_model.load_weights(weights_path)

In [12]:
validate_val_dataset(densenet161_predict_model)

Found 10095 images belonging to 2019 classes.
631/631 [==============================] - 118s 187ms/step
[1.8069624976275758, 0.5274888559105728]


[1.8069624976275758, 0.5274888559105728]

In [13]:
train_data_generator = DataGenerator(file_type='train', corp=True, batch_size=batch_size, shuffle=False)

Found 1008641 images belonging to 2019 classes.


In [15]:
validate_model(densenet161_predict_model, train_data_generator)

63041/63041 [==============================] - 18822s 299ms/step
[1.1865289402743124, 0.7364324868808625]


[1.1865289402743124, 0.7364324868808625]

In [ ]:
densenet161_filenames, densenet161_pred = predict(densenet161_predict_model, train_data_generator, steps=len(train_data_generator))

In [ ]:
np.save("./submissions/middle/densenet161_train_e01_1.8070_0.5275_pred.npy", densenet161_pred)
np.save("./submissions/middle/densenet161_train_e01_1.8070_0.5275_filenames.npy", densenet161_filenames)

In [16]:
densenet161_labels = train_data_generator.image_iterator.labels
np.save("./submissions/middle/densenet161_train_e01_1.8070_0.5275_labels.npy", densenet161_labels)

In [17]:
import numpy as np
densenet161_idx_list=[]
for idx in range(len(densenet161_pred)):
    idx_list = np.argsort(densenet161_pred[idx])
    densenet161_idx_list.append(idx_list)
    if idx % 1000 == 0:
        print(idx)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [29]:
import gc

In [30]:
del densenet161_labels

In [31]:
gc.collect()

0

In [33]:
print(len(densenet161_idx_list))

1008641


In [34]:
densenet161_idx_list = np.array(densenet161_idx_list)

MemoryError: 

In [35]:
print(densenet161_idx_list[0])

[1658 1916 1871 ...  620  678    0]


In [19]:
np.save("./submissions/middle/densenet161_train_e01_1.8070_0.5275_idx_list.npy", densenet161_idx_list)

MemoryError: 

In [ ]:
densenet161_filenames = np.load("./submissions/middle/densenet161_train_e01_1.8070_0.5275_filenames_part1.npy")
densenet161_pred = np.load("./submissions/middle/densenet161_train_e01_1.8070_0.5275_pred_part1.npy")

In [ ]:
top_hit = {}
for pos in range(100):
    top_hit[pos+1] = {}
for idx in range(len(densenet161_pred)):
    if idx % 1000 == 0:
        print(idx)
    for pos in range(100):
        true_label = densenet161_labels[idx]
        if densenet161_idx_list[idx][-1-pos] == true_label:
            top_hit[pos+1][true_label] = top_hit[pos+1].get(true_label, 0) + 1

In [ ]:
hit_count = 0
for i in range(100):
    K = i+1
    hit_count = hit_count + sum(top_hit[K].values())
    print("Top-%d:" % K)
    print(hit_count*1.0/len(densenet161_pred))
    print('-' * 20)